# Step 2: Build Model

In this part, we will show how to build model for qfold

In [1]:
from molecular_model import energy_calculator
from molecular_model import structure_calculator

2023-09-26 16:51:04.938764: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 16:51:04.961027: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 16:51:04.961444: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 16:51:05.399661: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
protein_name = 'glycylglycine'
aminoacids = 'GG'
number_bits_to_discretize_angles_range = 4
protein_id = 0

input_filename = 'inputRotations'
output_filename = 'outputRotations'
precalculated_energies_path = '../protein-folding-data/precalculated_energies/'

basis = '6-31g'
energy_method = 'mp2'
method_rotations_generation = 'minifold'

model_path = "./models/"
window_size = 5
max_aa_length = 200
path_input_file = "../protein-folding-data/training_data/training_70.txt"
epochs_train_classical_folding_model = 100
batch_size_classical_folding_model = 32

ener_calculator = energy_calculator.Energy_Calculator(input_filename, output_filename, precalculated_energies_path, basis, energy_method)
struct_calculator = structure_calculator.Structure_Calculator(model_path, window_size, max_aa_length, path_input_file, epochs_train_classical_folding_model, batch_size_classical_folding_model)

In [3]:
# Load variables from notebook 1-qfold-prepare-data.ipynb. PLEASE execute this notebook before
%store -r atoms
%store -r backbone

min_energy_psi4 = ener_calculator.calculate_energy_of_rotations(atoms)

#Get initial structure of the protein to rotate from it
[atoms, initialization_stats] = struct_calculator.calculate_initial_structure(atoms, aminoacids, method_rotations_generation, backbone)

#Calculate all posible energies for the phi and psi angles
deltasJson = ener_calculator.calculate_all_deltas_of_rotations(atoms, aminoacids, min_energy_psi4, protein_name, number_bits_to_discretize_angles_range, method_rotations_generation, backbone)

# Add the stadistics about the precision of the initializator
deltasJson['initialization_stats'] = initialization_stats
ener_calculator.write_json(deltasJson, 'delta_energies', protein_name, number_bits_to_discretize_angles_range, method_rotations_generation)


## MINIFOLD initialization for protein structure ##

    ⬤ Loading knowledge model


2023-09-26 16:51:07.624777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-26 16:51:07.625139: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


    ⬤ Generating input values
    ⬤ Generating input for minifold prediction
1/1 [==============================] - 0s 271ms/step

PHI precision:  8.752681663463036 % phi mean real value:  1.5708134759948975 phi mean calculated value: -1.2958056
PSI precision:  19.711415098740225 % psi mean real value:  -2.6404247826789864 psi mean calculated value: -0.1180845 

[-1.2958056] [-0.1180845]
    ⬤ Calculating energies for all posible rotations


Calculating energy of each position: 100%|██████████| 256/256 [06:53<00:00,  1.61s/it]

    ⬤ Calculating deltas for all possible combinations of rotations
